In [65]:
from sqlalchemy import create_engine
import pandas as pd
import plotly

import os
os.chdir(os.pardir)

from config import Config

primary_fuel_cmap={
        "Coal": '#636EFA',
        "Oil": '#EF553B',
        "Gas": '#00CC96',
        "Petcoke": '#AB63FA'
}

In [66]:
engine = create_engine(Config.SQLALCHEMY_DATABASE_URI)

In [67]:
switches = {'emission':'co2_lbs', 'country_long':'United States of America'}


query = f"""
    SELECT
        a.primary_fuel,
        SUM(a.cum_{switches['emission']}) as emission
    FROM {Config.SCHEMA}.plant a
    WHERE a.country_long = '{switches['country_long']}'
    GROUP BY a.primary_fuel
    """

df = pd.read_sql(query, engine)

df['percent'] = df['emission'] / df['emission'].sum()

df.columns = ['Primary Fuel', f"{switches['emission']} Cumulative Emissions (lbs)", "Percent"]

fig = px.pie(df, values='Percent', names='Primary Fuel', color_discrete_map=primary_fuel_cmap,
             title=f"Percent of {switches['country_long']} Power Sector Emissions by Fuel Source")
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()


In [85]:
switches = {'emission':'co2_lbs', 'country_long':'United States of America'}
switches['emission_label'] = f"{switches['emission'].replace('_lbs','').upper()} Cumulative Emissions (lbs)"

if switches['country_long'] == 'World':
    query = f"""
            SELECT
                a.plant_id_wri,
                a.country_long, 
                a.primary_fuel,
                a.cum_{switches['emission']},
                a.capacity_mw
            FROM {Config.SCHEMA}.plant a
            ORDER BY a.cum_{switches['emission']} DESC
            LIMIT 500
            """
else:
    query = f"""
        SELECT
            a.plant_id_wri,
            a.country_long, 
            a.primary_fuel,
            a.cum_{switches['emission']},
            a.capacity_mw
        FROM {Config.SCHEMA}.plant a
        WHERE a.country_long = '{switches['country_long']}'
        ORDER BY a.cum_{switches['emission']} DESC
        LIMIT 500
        """

df = pd.read_sql(query, engine)
df.columns = ['WRI Plant ID', 'Country', 'Primary Fuel', switches['emission_label'], "Plant Capacity MW"]

fig = px.scatter(df, x="Plant Capacity MW", y=switches['emission_label'],
                color='Primary Fuel', size=switches['emission_label'], opacity=0.5,
                hover_data=['Country'],
                marginal_x='histogram', marginal_y='histogram', color_discrete_map=primary_fuel_cmap)

fig.update_layout(xaxis_title='Capacity (MW)', yaxis_title=switches['emission_label'])
fig.update_layout(showlegend=False)



In [109]:
switches = {'emission':'co2_lbs', 'country_long':'United States of America'}
switches['emission_label'] = f"Mean {switches['emission'].replace('_lbs','').upper()} Emissions lbs/MWh"

if switches['country_long'] == 'World':
    query = f"""
            SELECT
                a.plant_id_wri,
                a.country_long, 
                a.primary_fuel,
                a.{switches['emission']}_per_mwh,
                a.cum_load
            FROM {Config.SCHEMA}.plant a
            ORDER BY a.cum_{switches['emission']} DESC
            LIMIT 500
            """
else:
    query = f"""
        SELECT
            a.plant_id_wri,
            a.country_long, 
            a.primary_fuel,
            a.{switches['emission']}_per_mwh,
            a.cum_load
        FROM {Config.SCHEMA}.plant a
        WHERE a.country_long = '{switches['country_long']}'
        ORDER BY a.cum_{switches['emission']} DESC
        LIMIT 500
        """

df = pd.read_sql(query, engine)

# --- clean up ---
df = df.dropna()
percentile = df[f"{switches['emission']}_per_mwh"].quantile(0.94)
df = df.loc[(df[f"{switches['emission']}_per_mwh"] > 0) & (df["cum_load"] > 0) & (df[f"{switches['emission']}_per_mwh"] < percentile)]

df.columns = ['WRI Plant ID', 'Country', 'Primary Fuel', switches['emission_label'], "Plant Cumulative Load (MWh)"]

fig = px.scatter(df, x="Plant Cumulative Load (MWh)", y=switches['emission_label'],
                color='Primary Fuel', size=switches['emission_label'], opacity=0.5,
                hover_data=['Country'],
                marginal_x='histogram', marginal_y='histogram', color_discrete_map=primary_fuel_cmap)

fig.update_layout(xaxis_title='Plant Cumulative Load (MWh)', yaxis_title=switches['emission_label'])
fig.update_layout(showlegend=False)

In [117]:
query = f"""
    SELECT
        a.country_long,
        a.cum_{switches['emission']}
    FROM {Config.SCHEMA}.country a
"""

df = pd.read_sql(query, engine)

df.columns = ['Country', f"{switches['emission_label']}"]

fig = px.pie(df, values=f"{switches['emission_label']}", names='Country',
            title=f"Power Sector {switches['emission_label']} by Country")
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(showlegend=False)

In [105]:
df['Mean CO2 Emissions lbs/MWh'].quantile(0.95)

67355.52872480165